<a href="https://colab.research.google.com/github/LuisFreire50/Banco-de-Dados/blob/main/C%C3%B3pia_de_Back_Mandante_baseado_em_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas scikit-learn numpy

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Carregar o dataset
df = pd.read_csv('https://www.football-data.co.uk/new/BRA.csv')

# Print the actual column names in the DataFrame
print(df.columns)

# Filtrar colunas relevantes (você pode ajustar conforme necessário)
# Ensure the column names match the actual column names in the DataFrame
# You may need to adjust the casing or remove extra whitespace
cols_to_use = ['Home', 'Away', 'PSCH', 'PSCD', 'PSCA', 'HG', 'AG']
df = df[cols_to_use]

# Codificar a variável alvo ('FTR' não está disponível, vamos inferir a partir dos gols)
df['resultado'] = df['HG'] > df['AG']  # True se home team venceu, False caso contrário
y = df['resultado'].astype(int)  # Convertendo para 0 ou 1

# Preparação das features
X = df.drop('resultado', axis=1)

# Codificar times como números
le = LabelEncoder()
X['Home'] = le.fit_transform(X['Home'])
X['Away'] = le.fit_transform(X['Away'])

# Dividir os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Index(['Country', 'League', 'Season', 'Date', 'Time', 'Home', 'Away', 'HG',
       'AG', 'Res', 'PSCH', 'PSCD', 'PSCA', 'MaxCH', 'MaxCD', 'MaxCA', 'AvgCH',
       'AvgCD', 'AvgCA', 'BFECH', 'BFECD', 'BFECA'],
      dtype='object')


In [ ]:
# Inicializar e treinar o modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Fazer previsões
y_pred = rf_model.predict(X_test)

# Avaliar o modelo
print(f'Acurácia: {accuracy_score(y_test, y_pred)}')
print('\nRelatório de Classificação:')
print(classification_report(y_test, y_pred))

Acurácia: 0.9989878542510121

Relatório de Classificação:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       497
           1       1.00      1.00      1.00       491

    accuracy                           1.00       988
   macro avg       1.00      1.00      1.00       988
weighted avg       1.00      1.00      1.00       988



In [ ]:
# Importância das features
feature_importance = pd.DataFrame({'feature': X.columns, 'importance': rf_model.feature_importances_})
print(feature_importance.sort_values('importance', ascending=False))

  feature  importance
5      HG    0.522981
6      AG    0.344342
4    PSCA    0.039289
2    PSCH    0.038913
3    PSCD    0.024514
0    Home    0.015082
1    Away    0.014879


In [ ]:
# Suponha que você tenha uma lista de jogos futuros
jogos_futuros = [
    {'Home': 'Flamengo RJ', 'Away': 'Fluminense', 'PSCH': 1.5, 'PSCD': 3.5, 'PSCA': 5.0, 'HG': 0, 'AG': 0},
    {'Home': 'Corinthians', 'Away': 'Sao Paulo', 'PSCH': 2.0, 'PSCD': 3.0, 'PSCA': 3.5, 'HG': 0, 'AG': 0},
    # Adicione mais jogos conforme necessário
]

# Transformar a lista em DataFrame
df_jogos_futuros = pd.DataFrame(jogos_futuros)

# Instead of directly setting le.classes_, fit the LabelEncoder
# with the unique values from both the training data and future data
all_teams = list(X['Home'].unique()) + list(X['Away'].unique()) + list(df_jogos_futuros['Home'].unique()) + list(df_jogos_futuros['Away'].unique())
le = LabelEncoder()
le.fit(all_teams)

df_jogos_futuros['Home'] = le.transform(df_jogos_futuros['Home'])
df_jogos_futuros['Away'] = le.transform(df_jogos_futuros['Away'])

# Selecionar as mesmas colunas usadas para treinar o modelo
X_futuro = df_jogos_futuros[['Home', 'Away', 'PSCH', 'PSCD', 'PSCA', 'HG', 'AG']]

In [ ]:
# Supondo que você já tenha treinado e guardado seu modelo como 'rf_model'
previsoes = rf_model.predict(X_futuro)
probabilidades = rf_model.predict_proba(X_futuro)

# Adicionar as previsões e probabilidades ao DataFrame original
df_jogos_futuros['previsao_vitoria_mandante'] = previsoes
df_jogos_futuros['prob_vitoria_mandante'] = probabilidades[:, 1]  # Probabilidade de vitória do mandante

# Mostrar os resultados
print(df_jogos_futuros[['Home', 'Away', 'previsao_vitoria_mandante', 'prob_vitoria_mandante']])

# Ordenar por probabilidade de vitória do mandante para facilidade de visualização
print(df_jogos_futuros.sort_values('prob_vitoria_mandante', ascending=False))

   Home  Away  previsao_vitoria_mandante  prob_vitoria_mandante
0    37    38                          0                   0.09
1    36    39                          0                   0.08
   Home  Away  PSCH  PSCD  PSCA  HG  AG  previsao_vitoria_mandante  \
0    37    38   1.5   3.5   5.0   0   0                          0   
1    36    39   2.0   3.0   3.5   0   0                          0   

   prob_vitoria_mandante  
0                   0.09  
1                   0.08  
